<p> Notice: This notebook is not optimized for memory nor performance yet. Please use it with caution when handling large datasets.


### Notice: just a tool for transforming between file types

---

## File reading

In [ ]:
# ! pip install fastparquet

In [ ]:
# ! pip install pyarrow

In [3]:
# Reading the saved dtypes Series
df_dtypes = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a_dtypes_series.csv'\
            , header=None, index_col=0, squeeze=True)
del df_dtypes.index.name
df_dtypes = df_dtypes.to_dict()

#### Read-in the file, either from .csv or .parquet

In [4]:
df = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a.csv'\
           , dtype= df_dtypes)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 3793 entries, SK_ID_CURR to compare_mean_IR_to_6_8y_recip_na
dtypes: float32(1720), float64(1), int32(386), int64(7), uint8(1679)
memory usage: 3.4 GB


In [6]:
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 3793 entries, SK_ID_CURR to compare_mean_IR_to_6_8y_recip_na
dtypes: float32(1720), float64(1), int32(386), int64(7), uint8(1679)
memory usage: 3.4 GB


In [7]:
# export df to parquet, do it once for each csv
df.to_parquet('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a.parquet.gzip' ,compression='gzip')

In [ ]:
df = pd.read_parquet('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a.parquet.gzip', engine = 'pyarrow')

In [ ]:
df.info()

In [ ]:
# df = df.astype(dtype=df_dtypes, copy=False)

In [ ]:
# df.info()

---

## Randomized sampleing:

#### If the dataset is too large, consider following randomized sampling from original dataset to facilitate development and testing

In [ ]:
# Randomized sampling from original dataset.
# This is just for simplifying the development process
# After coding is complete, should replace all df-->df, and remove this cell
# Reference: https://yiidtw.github.io/blog/2018-05-29-how-to-shuffle-dataframe-in-pandas/

df= df.sample(n = 1000).reset_index(drop=True)
df.shape

In [ ]:
df.head()

---

## Export to CSV

In [ ]:
# del df
# gc.collect()
# df.info()

In [ ]:
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
df.info()

In [ ]:
# Export the dtypes Series to csv for future use
df.dtypes.to_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_pyspark_sampled_G3V3_ohe_recip_dtypes_series.csv', header = False)

In [ ]:
# Export the dataframe to csv for future use
df.to_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_pyspark_sampled_G3V3_ohe_recip.csv', index = False)

---

# BayesianOptimization

https://github.com/fmfn/BayesianOptimization

In [ ]:
# ! pip install bayesian-optimization

In [ ]:
# conda install -c conda-forge bayesian-optimization
# conda install -c conda-forge xgboost

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
# from xgboost import XGBClassifier

In [ ]:
lgb.__version__

In [ ]:
# # Reading the saved dtypes Series
# df_dtypes = \
# pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V2_ohe_recip_20200216a_dtypes_series.csv'\
#             , header=None, index_col=0, squeeze=True)
# del df_dtypes.index.name
# df_dtypes df_dtypes.to_dict()

# df = \
# pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V2_ohe_recip_20200216a.csv'\
#            , dtype= df_dtypes)

In [ ]:
# df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]
# df.info()

In [ ]:
df['TARGET'].value_counts()

In [ ]:
# scale_pos_weight 
282686 / 24825

---

## BayesianOptimization- LGBM

In [ ]:
# ! pip install setuptools wheel numpy scipy "scikit-learn<=0.21.3" -U

In [ ]:
# ! pip install lightgbm

In [ ]:
del bo
gc.collect()

In [ ]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['max_bin'] = int(params['max_bin'])
#     params['max_drop'] = int(params['max_drop'])
    
        
    clf = LGBMClassifier(**params, 
#                          n_estimators = 2000,
#                          nthread = 2, 
                         boosting_type='goss',
#                          drop_seed = 924,
                         objective='binary',
                         scale_pos_weight = 11.387150050352467,
                         random_state = 924,
                         n_jobs = 6,
                         silent = False,
#                          importance_type (string, optional (default='split')) – 
#                          The type of feature importance to be filled into feature_importances_. 
#                          If ‘split’, result contains numbers of times the feature is used in a model. 
#                          If ‘gain’, result contains total gains of splits which use the feature.
                        )

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 200)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = { 
          'num_leaves': (7, 161), 
          'max_depth': (7, 189),
          'learning_rate': (.01, .1),
#           'n_estimators':(50, 1000),
#           'subsample_for_bin':(50000, 1000000),
#           'top_rate':(0.0 ,1.0),
          'min_split_gain': (0.01, 100),
#           'drop_rate': (0.0, 1.0),
#           'max_drop': (28, 343),
#           'skip_drop': (0.0, 1.0),
          'min_child_weight': (0.01, 100),
          'min_child_samples': (31, 999),
#         subsample (float, optional (default=1.)) – Subsample ratio of the training instance.
#         subsample_freq (int, optional (default=0)) – Frequence of subsample, <=0 means no enable.
#         colsample_bytree (float, optional (default=1.)) – Subsample ratio of columns when constructing each tree.
          'reg_alpha': (.00, 10.0), 
          'reg_lambda': (.00, 10.0), 
          'max_bin': (63, 511)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 21, n_iter = 21)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

In [ ]:
params_list = bo.res
len(params_list)

In [ ]:
# 輸出參數檔案
import json
with open('./params_list_BayesOpt_20200220a_G3V3_le_recip_lgbm.txt', 'w', encoding='utf-8') as fout:
    for params in params_list:
        json.dump(params, fout) 
        fout.write("\n")

In [ ]:
bo.max

In [ ]:
params_list

In [ ]:
params_list[34]

In [ ]:
params_list[19]

In [ ]:
# params_list[24]

In [ ]:
# no need for looping
# params_list = [bo.res[i] for i in range(10)]

---

In [ ]:
# 讀取參數檔案
with open('../../../BDSE12-Group3/datasets/homecdt_ss_output/params_list_BayesOpt_20200210a.txt', 'r', encoding='utf-8') as f:
    params_list_read = list(map(json.loads,f))

In [ ]:
params_list_read[8]

In [ ]:
type(params_list_read[8])

#### ========The end of BayesianOptimization========

---

## Interface connecting fteng & model parts

In [ ]:
# Assign df to final_df for follow-up modeling
final_df = df

# Apply the following gc if memory is running slow
del df
gc.collect()

In [ ]:
final_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]

In [ ]:
final_df.info()

---

## Modeling part. If using a ready dataset, please start here

In [ ]:
# Reading the saved dtypes Series
final_df_dtypes = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a_dtypes_series.csv'\
            , header=None, index_col=0, squeeze=True)
del final_df_dtypes.index.name
final_df_dtypes = final_df_dtypes.to_dict()

In [ ]:
final_df = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V3_le_recip_20200217a.csv'\
           , dtype= final_df_dtypes)

In [ ]:
final_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]
final_df.info()

This following is based on 'bear_Final_model' released 2020/01/23

In [2]:
# Forked from excellent kernel : https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features
# From Kaggler : https://www.kaggle.com/jsaguiar
# Just added a few features so I thought I had to make release it as well...

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import csv

In [ ]:
lgb.__version__

In [ ]:
print(final_df['TARGET'].isna().sum(), 
      final_df['TARGET'].dtypes)

In [ ]:
final_df['TARGET'].isnull().sum(), \
final_df['TARGET'].size, \
(final_df['TARGET'].isnull().sum()/final_df['TARGET'].size).round(4)

In [ ]:
final_df['TARGET'].value_counts()

In [ ]:
# 'scale_pos_weight'
282686/24825 

# LightGBM 模型

In [ ]:
# 讀取參數檔案
import json
with open('./params_list_BayesOpt_20200218b_G3V3_le_lgbm.txt', 'r', encoding='utf-8') as f:
    params_list_read = list(map(json.loads,f))

In [ ]:
# params_list_read
params_list_read[34]

In [ ]:
# params_list_read
params_list_read[19]

In [ ]:
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def kfold_lightgbm(df, num_folds = 5, stratified = True, debug= False, boosting_type= 'goss', epoch=20000, early_stop=200):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM goss. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=924)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=924)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        dtrain = lgb.Dataset(data=train_df[feats].iloc[train_idx], 
                             label=train_df['TARGET'].iloc[train_idx], 
                             free_raw_data=False, silent=True)
        dvalid = lgb.Dataset(data=train_df[feats].iloc[valid_idx], 
                             label=train_df['TARGET'].iloc[valid_idx], 
                             free_raw_data=False, silent=True)

        # LightGBM parameters found by Bayesian optimization

# # params_list_BayesOpt_20200218b_G3V3_le_lgbm.txt[19]
# {'target': 0.7901642737355099,
#  'params': {'learning_rate': 0.08661652062152753,
#   'max_bin': 501.5621837157406,
#   'max_depth': 62.953845761301494,
#   'min_child_samples': 772.7779165181825,
#   'min_child_weight': 22.929605988437682,
#   'min_split_gain': 18.34196792444013,
#   'num_leaves': 77.84272652383783,
#   'reg_alpha': 6.109027362545286,
#   'reg_lambda': 5.777795318140568}}

        params = {
            'learning_rate': 0.08661652062152753,
            'max_bin': 501,
            'max_depth': 63,
            'min_child_samples': 773, 
            'min_child_weight': 22.929605988437682,
            'min_split_gain': 18.34196792444013,            
            'num_leaves': 78,
            'reg_alpha': 6.109027362545286,
            'reg_lambda': 5.777795318140568,
            'objective': 'binary',
            'boosting_type': boosting_type,
            'nthread': 6,
            'scale_pos_weight': 11.387150050352467,
            'seed': 924,
            'verbose': 2000,
            'metric': 'auc',

#             'subsample': 0.9556663511637553,
#             'tree_learner': 'voting',
#             'colsample_bytree': 0.9497036,
#             'subsample_freq': 0,          
#             'histogram_pool_size': 20480
#             'device' : 'gpu',
#             'gpu_platform_id': 0,
#             'gpu_device_id':0
        }
        
        clf = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=epoch,
            valid_sets=[dtrain, dvalid],
            early_stopping_rounds=early_stop,
            verbose_eval=2000
        )

        oof_preds[valid_idx] = clf.predict(dvalid.data)
        sub_preds += clf.predict(test_df[feats]) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(dvalid.label, oof_preds[valid_idx])))
        del clf, dtrain, dvalid
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        sub_df = test_df[['SK_ID_CURR']].copy()
        sub_df['TARGET'] = sub_preds
        sub_df[['SK_ID_CURR', 'TARGET']].to_csv('./homecdt_submission_LGBM.csv', index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout
    plt.savefig('./lgbm_importances01.png')

## boosting_type：goss

In [ ]:
init_time = time.time()
kfold_lightgbm(final_df,10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

In [ ]:
init_time = time.time()
kfold_lightgbm(final_df,10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

## boosting_type：gbdt

In [ ]:
# init_time = time.time()
# kfold_lightgbm(final_df, 10, boosting_type= 'gbdt')
# print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

## boosting_type：dart

In [ ]:
# init_time = time.time()
# kfold_lightgbm(final_df,10, boosting_type= 'dart')
# print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

## boosting_type：rf

In [ ]:
# init_time = time.time()
# kfold_lightgbm(final_df,10,boosting_type= 'rf')
# print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

# XGBoost 模型

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle

In [ ]:
def kfold_xgb(df, num_folds, stratified = True, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting XGBoost. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1054)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1054)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        #if n_fold == 0: # REmove for full K-fold run
        cuda.select_device(0)
        cuda.close()
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf = XGBClassifier(learning_rate =0.01, 
                            n_estimators=5000, 
                            max_depth=4, 
                            min_child_weight=5,
#                             tree_method='gpu_hist',
                            subsample=0.8, 
                            colsample_bytree=0.8, 
                            objective= 'binary:logistic',
                            nthread=4,
                            scale_pos_weight=2.5,
                            seed=28,
                            reg_lambda = 1.2)
        
#         clf = pickle.load(open('test.pickle','rb'))
        
        cuda.select_device(0)
        cuda.close()
        
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 1000, early_stopping_rounds= 200)
        
        cuda.select_device(0)
        cuda.close()
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats])[:, 1] # / folds.n_splits # - Uncomment for K-fold 

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

        np.save("xgb_oof_preds_1", oof_preds)
        np.save("xgb_sub_preds_1", sub_preds)
        
        cuda.select_device(0)
        cuda.close()
        
    
    clf = pickle.load(open('test.pickle','rb'))
    # print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv('submission_XGBoost_GPU.csv', index= False)
    #display_importances(feature_importance_df)
    #return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('XGBoost Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('xgb_importances02.png')

In [ ]:
init_time = time.time()
kfold_xgb(final_df, 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

---

Below not executed

## Balance the 'TARGET' column

In [ ]:
appl_all_processed_df['TARGET'].value_counts()

In [ ]:
balanceFactor = ((appl_all_processed_df['TARGET'].value_counts()[0])/(appl_all_processed_df['TARGET'].value_counts()[1])).round(0).astype(int)
balanceFactor
# appl_all_processed_df['TARGET'].value_counts()[0]
# appl_all_processed_df['TARGET'].value_counts()[1]

In [ ]:
default_df = appl_all_processed_df[appl_all_processed_df['TARGET']==1]
default_df.shape

In [ ]:
default_df_balanced = pd.concat( [default_df] * (balanceFactor - 1), sort=False, ignore_index=True )
default_df_balanced.shape

In [ ]:
appl_all_processed_df_balanced = pd.concat([appl_all_processed_df , default_df_balanced], sort=False, ignore_index=True)
appl_all_processed_df_balanced.shape

In [ ]:
(appl_all_processed_df_balanced['TARGET'].unique(),
(appl_all_processed_df_balanced['TARGET'].value_counts()[1], \
appl_all_processed_df_balanced['TARGET'].value_counts()[0], \
appl_all_processed_df_balanced['TARGET'].isnull().sum()))

In [ ]:
# Apply the following gc if memory is running slow
del appl_all_processed_df_balanced
gc.collect()

---

---

# Todo

Todo:
* cleaning:
    * num_df: normalize with z-score
* feature engineering:
    * make reciprocol, polynomial columns of the existing columns. 1/x, x^x.
    * multiplying each columns, two columns at a time.
    * asset items, income items, willingness(history + misc profile) items, loading(principle + interest) items
    * Integration from other tables?

https://ithelp.ithome.com.tw/articles/10202059
https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
https://www.kaggle.com/parasjindal96/how-to-normalize-dataframe-pandas
    

---

## EDA

### Quick check for numerical columns

In [ ]:
numcol = df['CNT_FAM_MEMBERS']

In [ ]:
numcol.describe(), \
numcol.isnull().sum(), \
numcol.size

In [ ]:
numcol.value_counts(sort=True), numcol.unique().size

In [ ]:
# numcol_toYear = pd.to_numeric(\
#                               ((numcol.abs() / 365) \
#                                .round(0)) \
#                               ,downcast='integer')
# numcol_toYear.describe()

In [ ]:
# numcol_toYear.value_counts(sort=True), numcol_toYear.unique().size

### Quick check for categorical columns

In [ ]:
catcol = df['HOUR_APPR_PROCESS_START']

In [ ]:
catcol.unique(), \
catcol.unique().size

In [ ]:
catcol.value_counts(sort=True)

In [ ]:
catcol.isnull().sum(), \
catcol.size

In [ ]:
catcol.isnull().sum(), \
catcol.size

## Appendix

### Tool: Getting summary dataframe

In [ ]:
# might not be very useful at this point
def summary_df (data_df):
    """ 
    Output: a new dataframe with summary info from the input dataframe.
    Input: data_df, the original dataframe
    """
    summary_df = pd.concat([(data_df.describe(include='all')), \
           (data_df.dtypes.to_frame(name='dtypes').T), \
           (data_df.isnull().sum().to_frame(name='isnull').T), \
           (data_df.apply(lambda x:x.unique().size).to_frame(name='uniqAll').T)])
    return summary_df

def data_quality_df (data_df):
    """ 
    Output: a new dataframe with summary info from the input dataframe.
    Input: data_df, the original dataframe
    """
    data_quality_df = pd.concat([(data_df.describe(include='all')), \
           (data_df.dtypes.to_frame(name='dtypes').T), \
           (data_df.isnull().sum().to_frame(name='isnull').T), \
           (data_df.apply(lambda x:x.unique().size).to_frame(name='uniqAll').T)])
    return data_quality_df.iloc[[11,13,12,0,],:]


In [ ]:
data_quality_df(appl_all_df)

In [ ]:
# df.to_csv(file_name, encoding='utf-8', index=False)
# data_quality_df(df).to_csv("./eda_output/application_train_data_quality.csv")

In [ ]:
df['CNT_CHILDREN'].value_counts()

In [ ]:
df['CNT_CHILDREN'].value_counts().sum()

In [ ]:
df.describe()

In [ ]:
summary_df(df)

In [ ]:
# df.to_csv(file_name, encoding='utf-8', index=False)
# summary_df(df).to_csv("./eda_output/application_train_summary_df.csv")

---

### .nunique() function

In [ ]:
# nunique() function excludes NaN 
# i.e. it does not consider NaN as a "value", therefore NaN is not counted as a "unique value"
df.nunique()

In [ ]:
df.nunique() == df.apply(lambda x:x.unique().shape[0])

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].unique().shape[0]

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].nunique()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].unique().size

### .value_counts() function

In [ ]:
# .value_counts() function has similar viewpoint towards NaN.
# i.e. it does not consider null as a value, therefore not counted in .value_counts()

In [ ]:
df['NAME_TYPE_SUITE'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].size

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].value_counts().sum() + df['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum() == \
df['AMT_REQ_CREDIT_BUREAU_YEAR'].size

### 重複值

In [ ]:
# Counting unique values (cf. .nunique() function, see above section)
# This code was retrieved from HT

df.apply(lambda x:x.unique().shape[0])

In [ ]:
# It is the same if you write (df.apply(lambda x:x.unique().size))
assert (df.apply(lambda x:x.unique().shape[0])==df.apply(lambda x:x.unique().size)).all

In [ ]:
# # %timeit showed the performances are similar
# %timeit df.apply(lambda x:x.unique().shape[0])
# %timeit df.apply(lambda x:x.unique().size)

### 空值

In [ ]:
# 含空值欄位占比
print(f"{df.isnull().any().sum()} in {df.shape[1]} columns (ratio: {(df.isnull().any().sum()/df.shape[1]).round(2)}) has empty value(s)")


---

## re-casting to reduce memory use (beta)

In [ ]:
# np.isfinite(num_df).all().value_counts()

In [ ]:
# num_df_finite = num_df[num_df.columns[np.isfinite(num_df).all()]]
# num_df_infinite = num_df[num_df.columns[np.isfinite(num_df).all() == False]]
# num_df_finite.shape, num_df_infinite.shape

In [ ]:
# assert num_df_finite.shape[0] == num_df_infinite.shape[0] == num_df.shape[0]
# assert num_df_finite.shape[1] + num_df_infinite.shape[1] == num_df.shape[1]

In [ ]:
# def reduce_mem_usage(props, finite:bool = True):
#     props.info(verbose=False)
#     start_mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage of properties dataframe is :",start_mem_usg," MB")
#     if finite == True:  
#         props[props.columns[(props.min()>=0) & (props.max()<255)]] = \
#         props[props.columns[(props.min()>=0) & (props.max()<255)]].astype(np.uint8, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 255) & (props.max()<65535)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 255) & (props.max()<65535)]] \
#         .astype(np.uint16, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 65535) & (props.max()<4294967295)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 65535) & (props.max()<4294967295)]] \
#         .astype(np.uint32, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 4294967295)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 4294967295)]] \
#         .astype(np.uint64, copy=False)
#         props.info(verbose=False)
#     else:
#         props = props.astype(np.float32, copy=False)
#         props.info(verbose=False)
        
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
#     return props

# if num_na_df_finite.min()>=0:
#     if num_na_df_finite.max() < 255:
#         props[col] = props[col].astype(np.uint8)
#     elif num_na_df_finite.max() < 65535:
#         props[col] = props[col].astype(np.uint16)
#     elif num_na_df_finite.max() < 4294967295:
#         props[col] = props[col].astype(np.uint32)
#     else:
#         props[col] = props[col].astype(np.uint64)

In [ ]:
# num_df_finite.info()

In [ ]:
# num_df_finite = reduce_mem_usage(num_df_finite, finite = True)

In [ ]:
# num_df_infinite.info()

In [ ]:
# num_df_infinite = reduce_mem_usage(num_df_infinite, finite = False)

In [ ]:
# num_df = pd.concat([num_df_finite, num_df_infinite], axis ='columns')
# num_df.info()

In [ ]:
# assert num_df_finite.shape[0] == num_df_infinite.shape[0] == num_df.shape[0]
# assert num_df_finite.shape[1] + num_df_infinite.shape[1] == num_df.shape[1]

In [ ]:
# del num_df_finite
# del num_df_infinite
# gc.collect()

THE END